<a href="https://colab.research.google.com/github/amengemeda/ISproject-2/blob/main/Improved_Amharic_Hate_Speech_detection_using_fine_tuned_mBERT_(Trainer_API).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tuning BERT for the downstream task of detecting hate speech on Amharic texts

In [ ]:
#installing the transformers package
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.9 MB/s 
     |████████████████████████████████| 7.6 MB 43.6 MB/s 
     |████████████████████████████████| 182 kB 60.9 MB/s 


In [ ]:
#installing the datasets package
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.0 MB/s 
     |████████████████████████████████| 132 kB 57.3 MB/s 
     |████████████████████████████████| 212 kB 61.7 MB/s 
     |████████████████████████████████| 127 kB 59.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
#importing the datasets package
from datasets import Dataset
import datasets
#import load metric for model evaluation
from datasets import load_metric

In [ ]:
#import numpy and pandas for mathematical computation and data manipulation respectively 
import numpy as np
import pandas as pd
#import drive package to connect this colab file with the drive where the data will be retrived from
from google.colab import drive
#import the pipeline of transformers
from transformers import pipeline
#import AutoTokenizer for tokenization purposes
from transformers import AutoTokenizer
#import the Trainer API
from transformers import TrainingArguments, Trainer
#import early stopping callback 
from transformers import EarlyStoppingCallback, IntervalStrategy


In [ ]:
#import torch
import torch
#import Data loader from torch
from torch.utils.data import DataLoader
#import an optimizer
from torch.optim import AdamW
#import tqdm for a progress bar
from tqdm.auto import tqdm

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#mount google drive to access the dataset directly from the drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
#fetch the dataset from the drive
Labels=pd.read_csv('/content/drive/MyDrive/Machine Learning/Data/Amharic-Hate-Speech-Dataset/Labels.txt',header=None)
Posts=pd.read_csv('/content/drive/MyDrive/Machine Learning/Data/Amharic-Hate-Speech-Dataset/Posts.txt',header=None)

In [ ]:
#naming the columns
Labels.columns = ["labels"]
Posts.columns = ["post"]

In [ ]:
#encoding the classes into numerical data
Labels = Labels.replace(['Free', 'Free ','Hate'],[0,0,1]) 

In [ ]:
#check the encoded label data
Labels.head(10)

,labels
0,0
1,0
2,0
3,0
4,1
5,0
6,0
7,0
8,1
9,0


In [ ]:
#check the Amharic data
Posts.head(1000)

,post
0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...,...
995,መፈንቅለመንግስት ከሽፏልእንዴ የየክልሉ ህዝብ ምን ይጠብቃል ሆ ብሎ ሄዶ ...
996,ሰው በሩን እንደፈለገ መክፈት እንደፈለገ መዝጋት ይችላል የምን አውቅልሻ...
997,ክክክክክ የሱዳን ዜጋ ናችሁ እንዴ ባለ ነጠላ ጎጃሜ ሁላ
998,የምን ማለቃቀስ ነው ምድረ ፎካሪ ሁላ


In [ ]:
#merge the datasets
Frames = [Labels, Posts]
Merged = pd.concat(Frames, axis=1)


In [ ]:
#preview of merged data
Merged

,labels,post
0,0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,0,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,0,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,0,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,1,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...,...,...
29995,1,በአሉ የሁሉም ኢትዮጵያዊ ስላልሆነ በኦሮምኛው ቢለፋደድ ምን አገባን
29996,0,ተባረክ አብቹ ፈር ቀዳጅ ስለሆንህ መጋረጃው መቀደድ ስለጀመረ
29997,0,እስከ አሁን አንተ ብቻ ነው በ መፅሀፍ ያልቻልከው አንተም ታሪክ እ...
29998,1,ህገወጥት ጠቅላይ ሚንስትር ፅቤት የተፈቀደ ሆኖ ህዝብን እንዴት ህግ አክ...


In [ ]:
train_val_df, test_dataset = train_test_split(Merged, test_size=0.20, random_state=42)
train_dataset, evaluation_dataset = train_test_split(train_val_df, test_size=0.115, random_state=42)
print('Training dataset shape: ', train_dataset.shape)
print('Validation dataset shape: ', evaluation_dataset.shape)
print('Testing dataset shape: ', test_dataset.shape)

Training dataset shape:  (21240, 2)
Validation dataset shape:  (2760, 2)
Testing dataset shape:  (6000, 2)


In [ ]:
#Divide the dataset into train and test categories 
msk = np.random.rand(len(Merged)) < 0.8
train_dataset = Merged[msk]
test_dataset = Merged[~msk]

In [ ]:
#convert format of the dataset to HuggingFace Dataset from Pandas DataFrame
test_dataset=Dataset.from_pandas(test_dataset)

In [ ]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
train_dataset=Dataset.from_pandas(train_dataset)

In [ ]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
evaluation_dataset=Dataset.from_pandas(evaluation_dataset)

In [ ]:
#preview of the dataset after conversion
(test_dataset)

Dataset({
    features: ['labels', 'post', '__index_level_0__'],
    num_rows: 5940
})

In [ ]:
#remove unnecessary column
test_dataset=test_dataset.remove_columns("__index_level_0__")
train_dataset=train_dataset.remove_columns("__index_level_0__")
evaluation_dataset=evaluation_dataset.remove_columns("__index_level_0__")

In [ ]:
#combine the train and test dataset into one datset
main_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'evaluate': evaluation_dataset
})

In [ ]:
#preview of the dataset after merging
main_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'post'],
        num_rows: 24060
    })
    test: Dataset({
        features: ['labels', 'post'],
        num_rows: 5940
    })
    evaluate: Dataset({
        features: ['labels', 'post'],
        num_rows: 2760
    })
})

In [ ]:
# training and testing data size
training_data_size = main_dataset['train'].num_rows
testing_data_size = main_dataset['test'].num_rows
evaluation_data_size = main_dataset['evaluate'].num_rows

In [ ]:
#loading a tokenizer from the pretrained model
tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic")

Downloading:   0%|          | 0.00/335 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
#Have a tokenizer function that uses the tokenizer 
def tokenize_function(data):
    return tokenizer(data["post"], padding="max_length", truncation=True)

In [ ]:
#Tokenize all the data using the mapping functionality
tokenized_datasets = main_dataset.map(tokenize_function)


  0%|          | 0/24060 [00:00<?, ?ex/s]

  0%|          | 0/5940 [00:00<?, ?ex/s]

  0%|          | 0/2760 [00:00<?, ?ex/s]

In [ ]:
#empty cache
torch.cuda.empty_cache()

In [ ]:
#remove the posts column as it is no longer needed
tokenized_datasets = tokenized_datasets.remove_columns(["post"])

In [ ]:
#changing the format of the tokenized dataset to torch
tokenized_datasets.set_format("torch")

In [ ]:
#shuffeling and selecting the needed size of dataset for training and evaluating the model
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(training_data_size))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(testing_data_size))
small_eval_dataset = tokenized_datasets["evaluate"].shuffle(seed=42).select(range(evaluation_data_size))

In [ ]:
# preview of the shuffeled and selected evaluation dataset
small_eval_dataset

Dataset({
    features: ['labels', 'post', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2760
})

In [ ]:
# preview of the shuffeled and selected training dataset
small_train_dataset

Dataset({
    features: ['labels', 'post', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 24060
})

In [ ]:
# preview of the shuffeled and selected testing dataset
small_test_dataset

Dataset({
    features: ['labels', 'post', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5940
})

In [ ]:
#load the dataset using DataLoader
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=4)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=4)
test_dataloader = DataLoader(small_test_dataset, batch_size=4)

In [ ]:
#Load auto mode classifier from the pretrained model 
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("Davlan/", num_labels=2)

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-amharic were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model check

In [ ]:

from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1","accuracy")

<ipython-input-37-a9d1bd3d149e>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1","accuracy")


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy = IntervalStrategy.STEPS, # "steps"
   warmup_steps=500,                # number of warmup steps for learning rate  
   save_steps=2000,
   eval_steps = 2000, # Evaluation and Save happens every 2000 steps
   save_total_limit = 3, # Only last 3 models are saved. Older ones are deleted.
   learning_rate=1e-5,
   per_device_train_batch_size=4,
   per_device_eval_batch_size=4,
   num_train_epochs=15,
   weight_decay=0.01,
   push_to_hub=False,
   metric_for_best_model = 'f1',
   do_predict=True,
   load_best_model_at_end=True)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: post. If post are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 24060
  Num Epochs = 15
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 90225
  Number of trainable parameters = 177854978


Step,Training Loss,Validation Loss,F1
2000,0.685100,0.715681,0.404950
4000,0.672800,0.619588,0.670356
6000,0.667800,0.617834,0.684302
8000,0.619000,0.617754,0.712747
10000,0.625800,0.560830,0.763150
12000,0.617100,0.501124,0.778472
14000,0.564900,0.539094,0.803781
16000,0.552400,0.498837,0.806076
18000,0.601000,0.452078,0.835222
20000,0.614000,0.548935,0.842802


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: post. If post are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2760
  Batch size = 4
Saving model checkpoint to training_with_callbacks/checkpoint-2000
Configuration saved in training_with_callbacks/checkpoint-2000/config.json
Model weights saved in training_with_callbacks/checkpoint-2000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: post. If post are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2760
  Batch size = 4
Saving model checkpoint to training_with_callbacks/checkpoint-4000
Configuration saved in training_with_callbacks/checkpo

TrainOutput(global_step=90225, training_loss=0.36669612528313955, metrics={'train_runtime': 44665.7688, 'train_samples_per_second': 8.08, 'train_steps_per_second': 2.02, 'total_flos': 9.4956779879424e+16, 'train_loss': 0.36669612528313955, 'epoch': 15.0})

In [ ]:
trainer.evaluate(small_test_dataset)

In [ ]:
#install huggingface_hub package to interact with huggingface platform
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login

In [ ]:
#login to huggingface
notebook_login()

In [ ]:
#specify the path for the trainde model and tokenizer to huggingface repository
model.save_pretrained("path/to/improved-amharic-hate-speech-detection-mBERT")
tokenizer.save_pretrained("path/to/improved-amharic-hate-speech-detection-mBERT")

In [ ]:
#push the trained model to huggingface repository
model.push_to_hub("improved-amharic-hate-speech-detection-mBERT")

In [ ]:
#push the tokenizer to huggingface repository
tokenizer.push_to_hub("improved-amharic-hate-speech-detection-mBERT")